## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - update_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
  - EtaslPlanner
    - pick, place, ~~sweep~~
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.02,0.02,0.1), center=(0,0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.02,0.02,0.1), center=(0,-0.06,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", rname="indy0", 
                     _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepPoint
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### update_state
* update_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [12]:
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [13]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

In [14]:
# pick motion - move the object to "gripper"
from_state = initial_state.copy(pscene)
to_node = ("gripper", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    pick_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

try transition motion
transition motion tried: False
try transition motion
transition motion tried: False
try transition motion
transition motion tried: False
try transition motion
transition motion tried: False
try transition motion
transition motion tried: True


In [15]:
# place motion - move the object to "goal"
from_state = pick_state.copy(pscene)
to_node = ("goal", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    place_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this placement motion is inherently impossible depending on pick parameter.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

try transition motion
transition motion tried: True


In [16]:
# sweep motion 1 - reach the first waypoint
from_state = place_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("goal", 1)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        if np.abs(LastQ[0])<np.pi/2: # else, it will fail in sweep motion
            break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    sweep1_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

try transition motion
transition motion tried: True


In [17]:
# sweep motion 2 - sweep "floor" to the second waypoint. Constrained motion planning takes longer time
from_state = sweep1_state.copy(pscene)
to_node = ("goal", 2)
for _ in range(3):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=30)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    sweep2_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this constrained motion is inherently impossible.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

try constrained motion
constrained motion tried: True


In [18]:
# return: calling sample_leaf_state() with same binding state generates motion target that returns to home pose
from_state = sweep2_state.copy(pscene)
to_node = ("goal", 2)
for _ in range(10):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    return_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

============= try go home (indy0) ===================
try joint motion
joint motion tried: True


##### EtaslPlanner
* same usage as the MoveitPlanner
- pick, place, ~~sweep~~

In [19]:
from pkg.planning.motion.etasl.etasl_planner import EtaslPlanner
mplan = EtaslPlanner(pscene)

In [20]:
# pick motion - move the object to "gripper"
from_state = initial_state.copy(pscene)
to_node = ("gripper", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
    pick_state_etasl = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [21]:
# place motion - move the object to "goal"
from_state = pick_state_etasl.copy(pscene)
to_node = ("goal", 0)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    print("Sometimes this placement motion is inherently impossible depending on pick parameter.")
    print("Then, please re-try from the beginning to change the configuration")
    raise(RuntimeError("Motion plan failure"))

In [22]:
# sweep motion 1 - reach the first waypoint
from_state = place_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("goal", 1)
for _ in range(20):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [23]:
# sweep motion 2 - sweep "floor" to the second waypoint.
# Constrained motion is not implemented with eTaSL planner. So, it will not "sweep" the floor
from_state = sweep1_state.copy(pscene)
to_node = ("goal", 2)
for _ in range(10):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

In [24]:
# return: calling sample_leaf_state() with same binding state generates go-home motion target
from_state = sweep2_state.copy(pscene)
to_node = ("goal", 2)
for _ in range(10):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=1)
    if success:
        break
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.05)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
    raise(RuntimeError("Motion plan failure"))

============= try go home (indy0) ===================
===================== plan joint manipulation =====================


## 4.3 TaskPlanner

##### create PlanningPipeline

In [25]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(MoveitPlanner(pscene))

##### RRT

In [26]:
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

- pick & place & sweep
  - single process

In [33]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->1 (3.11/100.0 s, steps/err: 35(46.3078022003 ms)/0.00151201111736)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: False
result: 0 - ('floor', 0)->('gripper', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (4.18/100.0 s, steps/err: 11(18.6560153961 ms)/0.00195372001577)


- pick & place & sweep  
  - multi process

In [34]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              display=False, dt_vis=0.01, timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=10)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.05)

Use 20/20 agents
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
transition motion tried: True
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
result: 0 - ('floor', 0)->('gripper', 0) = success
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (0.14/100.0 s, steps/err: 22(78.8321495056 ms)/0.00113651317018)
branching: 0->1 (0.14/100.0 s, steps/err: 20(44.196844101 ms)/0.000697609178836)
branching: 0->3 (0.18/100.0 s, steps/e

transition motion tried: True
transition motion tried: False
result: 3 - ('gripper', 0)->('goal', 0) = success
transition motion tried: True
transition motion tried: False
result: 3 - ('gripper', 0)->('goal', 0) = success
result: 0 - ('floor', 0)->('gripper', 0) = fail
branching: 3->18 (1.29/100.0 s, steps/err: 29(75.1168727875 ms)/0.00100573079929)
result: 2 - ('gripper', 0)->('floor', 0) = fail
branching: 3->17 (1.29/100.0 s, steps/err: 32(178.763151169 ms)/0.00176875809993)
try: 17 - ('goal', 0)->('goal', 1)
try: 18 - ('goal', 0)->('goal', 1)
try transition motion
try: 11 - ('gripper', 0)->('goal', 0)
try transition motion
try: 11 - ('gripper', 0)->('goal', 0)
try transition motion
try transition motion
transition motion tried: True
transition motion tried: True
=============== try reaching goal from ('goal', 1) =================
=============== try reaching goal from ('goal', 1) =================
transition motion tried: True
transition motion tried: True
result: 18 - ('goal', 0)->

======================= terminated 0: first answer acquired ===============================
constrained motion tried: True
result: 15 - ('goal', 1)->('goal', 2) = success
branching: 15->35 (8.88/100.0 s, steps/err: 46(7911.03100777 ms)/0.00180443414113)
++ adding return motion to acquired answer ++
try joint motion
joint motion tried: True
======================= terminated 14: first answer acquired ===============================
constrained motion tried: False
result: 7 - ('gripper', 1)->('gripper', 2) = fail
======================= terminated 17: first answer acquired from other agent ===============================
constrained motion tried: False
result: 7 - ('gripper', 1)->('gripper', 2) = fail
======================= terminated 2: first answer acquired from other agent ===============================
constrained motion tried: False
result: 10 - ('floor', 1)->('floor', 2) = fail
======================= terminated 12: first answer acquired from other agent =========================

======================= terminated 4: first answer acquired from other agent ===============================
